https://archive.ax.dev/versions/0.4.1/tutorials/multiobjective_optimization.html

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from botorch.test_functions.multi_objective import BraninCurrin,ZDT1,ZDT2,ZDT3
from complexity.glch_experiments_functions import save_glch_data



In [8]:
class BaseEvaluator:

    def __init__(self,name,best_init_pt,best_init_pt_img_loc,best_h1_list,best_h2_list,func):
        self.name = name
        self.best_init_pt = best_init_pt # best initial point
        self.best_init_pt_img_loc = best_init_pt_img_loc # best initial point image location
        self.best_h1_list = best_h1_list
        self.best_h2_list = best_h2_list
        self.func = func

    def evaluate_one(self,parameters):
        evaluation = self.func(torch.tensor([parameters.get("h1"), parameters.get("h2")]))
        return {"a": evaluation[0].item(), "b": evaluation[1].item()}

    def evaluate_many(self,possible_values):
        
        evaluations = []
        for h1 in possible_values["h1"]:
            for h2 in possible_values["h2"]:
                e = self.evaluate_one({"h1":h1/max(possible_values["h1"]),"h2":h2/max(possible_values["h2"])})
                evaluations.append({"h1":h1,"h2":h2,"a": e["a"],"b": e["b"]})
        
        data = pd.DataFrame(evaluations)
    
        data["topology"] = (data["h1"]).astype(int).apply(lambda x: f"{x:02d}") + "_" + \
            (data["h2"]).astype(int).apply(lambda x: f"{x:02d}")
    
        data = data.set_index("topology")
    
        return data


class BraninCurrinEvaluator(BaseEvaluator):

    def __init__(self):
        super().__init__(
            "branin_currin",
            [0,0],
            "right",
            list(np.arange(0,40 + 1)),
            list(np.arange(0,10 + 1)),
            BraninCurrin(negate=False).to(dtype=torch.double,device=torch.device("cpu"))
        )

class ZDT1Evaluator(BaseEvaluator):

    def __init__(self):
        super().__init__(
            "zdt1",
            [0,0],
            "left",
            list(np.arange(0,40 + 1)),
            list(np.arange(0,10 + 1)),
            ZDT1(2).to(dtype=torch.double,device=torch.device("cpu"))
        )

# Exploration

In [3]:
# evaluator = BraninCurrinEvaluator()
evaluator = ZDT1Evaluator()

In [4]:
data = evaluator.evaluate_many({"h1": list(np.arange(0,40 + 1)),"h2": list(np.arange(0,10 + 1))})

In [15]:
# data.loc["40_10"]

In [16]:
# data.loc["00_00"]

In [13]:
# import matplotlib
# plt.close('all')
# matplotlib.use("Qt5Agg")
# %matplotlib inline
# plt.scatter(list(data["a"]),list(data["b"]))

In [14]:
# plt.scatter(list(data["h1"]),list(data["h2"]))

# Experiment

In [9]:

def glch_synthetic_2d(evaluator):
    
    possible_values = {"h1": evaluator.best_h1_list,"h2": evaluator.best_h2_list} # possible values for the hyperparameters
    data = evaluator.evaluate_many(possible_values) # dataframe that maps hyperparameters to objectives
    to_str_method = lambda p : '_'.join(map(lambda x: f"{x:02d}",[p["h1"],p["h2"]])) # converts hyperparameters to dataframe index
    initial_values = {"h1":evaluator.best_init_pt[0],"h2":evaluator.best_init_pt[1]} # initial hyperparameters
    x_in_log_scale=False # used when plotting, when the x-axis varies too much, e.g., network parameters
    algo="glch" # options are glch (2d) and gho (1d)
    select_function="angle_rule" # Used when algo == glch. options are tie_break,gift_wrapping,angle_rule. Recommended: angle_rule
    constrained=True # Used when select_function == gift_wrapping/angle_rule. We only implemented the constrained tie_break select function
    fldr="glch_synthetic_data_results" # results are stored here
    debug=False # used to debug the algorithm implementation if any problem is detected
    debug_folder="glch_synthetic_data_debug" # debug results are stored here
    title = evaluator.name # identifies the experiment
    start=evaluator.best_init_pt_img_loc # if start == left, the algorithm finds the lch from left to right
    weights = [1,1] # used only in 1d problems
    axes = ["a","b"] # names of the objectives in the data variable
    axes_ranges=[None,None] # used only to adjust the plots
    axes_aliases=["1st Objective","2nd Objective"] # names of the objectives on the plots
    axes_scales = [data["a"].max() - data["a"].min(),data["b"].max() - data["b"].min()] # used only if select_function == tie_break
    
    
    save_glch_data(
        algo,
        data,possible_values,axes,initial_values,to_str_method,constrained,weights,start,
        axes_scales,
        debug,title,debug_folder,select_function,
        x_in_log_scale,axes_ranges,axes_aliases,fldr)

In [11]:
# glch_synthetic_2d(BraninCurrinEvaluator())

In [12]:
# glch_synthetic_2d(ZDT1Evaluator())